In [1]:
from sympy import symbols, simplify, Not, And, Or, Implies, satisfiable
from itertools import combinations
from functools import reduce
#libreria per il debugging
import pdb
%pdb on


Automatic pdb calling has been turned ON


In [4]:
# ============================================================
#  sd-DNNF compilation + Weighted Model Counting (SymPy)
#  Pipeline: formula -> NNF -> DNNF -> d-DNNF -> sd-DNNF -> WMC
# ============================================================

from sympy import symbols, And, Or, Not, simplify, S
from sympy.logic.boolalg import to_nnf
from sympy.logic.inference import satisfiable

# ---------------------------
# Shannon expansion
# ---------------------------
def shannon_expansion(f, s):
    """
    Shannon expansion of f on variable s:
        f == (s ∧ f|s=True) ∨ (¬s ∧ f|s=False)
    """
    if s not in f.free_symbols:
        return f
    f1 = f.subs({s: S.true})
    f0 = f.subs({s: S.false})
    return simplify(Or(And(s, f1), And(Not(s), f0)))

# ---------------------------
# Utilities for factorization
# ---------------------------
def _var_components(conj):
    """
    Given a conjunction (And(...)), partition factors into variable-connected components:
    factors that share variables end up in the same component.
    This helps build decomposable ANDs (children with disjoint var sets).
    """
    if isinstance(conj, And):
        factors = list(conj.args)
    else:
        factors = [conj]
    comps = []
    while factors:
        seed = factors.pop()
        group = [seed]
        vars_grp = set(seed.free_symbols)
        changed = True
        while changed:
            changed = False
            rest = []
            for f in factors:
                if vars_grp & set(f.free_symbols):
                    group.append(f)
                    vars_grp |= set(f.free_symbols)
                    changed = True
                else:
                    rest.append(f)
            factors = rest
        comps.append(group)
    return comps

# ---------------------------
# 1) NNF -> DNNF (decision-style)
# ---------------------------
def NNF2DNNF(nnf):
    """
    Compile NNF -> DNNF.
    - Se un AND si spezza in >1 componenti variabile-disgiunte, le compila e le congiunge (decomposable).
    - Se NON si spezza (1 sola componente), fai fallback a Shannon su una variabile pivot.
    - Gli OR si compilano ricorsivamente; la determinism la imponiamo dopo.
    """
    atoms = list(nnf.free_symbols)
    if len(atoms) <= 1:
        return simplify(nnf)

    if isinstance(nnf, And):
        groups = _var_components(nnf)
        if len(groups) > 1:
            compiled = [
                NNF2DNNF(And(*g)) if len(g) > 1 else NNF2DNNF(g[0])
                for g in groups
            ]
            return simplify(And(*compiled))  # decomposable AND
        else:
            # niente decomposizione possibile -> forza un cambiamento strutturale con Shannon
            pivot = atoms[0]
            return NNF2DNNF(shannon_expansion(nnf, pivot))

    if isinstance(nnf, Or):
        kids = [NNF2DNNF(k) for k in nnf.args]
        return simplify(Or(*kids))  # determinism verrà imposto dopo

    # fallback generale (in pratica non dovresti arrivarci con NNF pulita)
    pivot = atoms[0]
    return NNF2DNNF(shannon_expansion(nnf, pivot))


# ---------------------------
# 2) DNNF -> d-DNNF (enforce determinism on OR)
# ---------------------------
def _make_or_deterministic(expr):
    """
    Given Or(args), make it deterministic:
    repeatedly split (with Shannon) while two children overlap (i.e., their conjunction is satisfiable).
    """
    assert isinstance(expr, Or)
    args = list(expr.args)

    i = 0
    while i < len(args):
        j = i + 1
        while j < len(args):
            inter = And(args[i], args[j])
            if satisfiable(inter):
                # pick a pivot (try from the intersection; else from the union)
                inter_vars = list(inter.free_symbols)
                if inter_vars:
                    pivot = inter_vars[0]
                else:
                    union_vars = list((args[i] | args[j]).free_symbols)
                    pivot = union_vars[0] if union_vars else next(iter(expr.free_symbols))
                # split the whole OR on pivot and restart
                return _make_or_deterministic(shannon_expansion(Or(*args), pivot))
            j += 1
        i += 1
    return Or(*args)

def DNNF2dDNNF(dnnf):
    """
    Recursively ensure every OR is deterministic.
    AND remains as is (already decomposable from NNF2DNNF step).
    """
    if isinstance(dnnf, And):
        return And(*(DNNF2dDNNF(a) for a in dnnf.args))
    if isinstance(dnnf, Or):
        det_kids = Or(*(DNNF2dDNNF(a) for a in dnnf.args))
        return _make_or_deterministic(det_kids)
    return dnnf  # literal / True / False

# ---------------------------
# 3) Smoothing d-DNNF -> sd-DNNF
# ---------------------------
def _tau(v):
    return Or(v, Not(v))  # tautology mentioning v

def _vars(n):
    return set(n.free_symbols)

def ddnnf2sdNNF(ddnnf):
    """
    Make every OR smooth: all children mention the same variable set.
    Insert neutral tautologies (v ∨ ¬v) for missing variables in each child.
    """
    if isinstance(ddnnf, And):
        return And(*(ddnnf2sdNNF(a) for a in ddnnf.args))
    if isinstance(ddnnf, Or):
        kids = [ddnnf2sdNNF(a) for a in ddnnf.args]
        allv = set().union(*(_vars(k) for k in kids))
        smoothed = []
        for k in kids:
            missing = allv - _vars(k)
            kk = k
            for v in missing:
                kk = And(kk, _tau(v))
            smoothed.append(kk)
        return Or(*smoothed)
    return ddnnf  # literal / True / False

# ---------------------------
# 4) Weighted Model Counting on sd-DNNF
# ---------------------------
def model_counting_sdnf(sdNNF_formula, weights):
    """
    Compute WMC bottom-up on an sd-DNNF:
      - OR -> sum
      - AND -> product
      - literals -> weights[literal]
    'weights' must map SymPy literals to floats, e.g. {a:0.6, Not(a):0.4, ...}
    """
    if sdNNF_formula is S.true:
        return 1.0
    if sdNNF_formula is S.false:
        return 0.0
    if isinstance(sdNNF_formula, Or):
        return sum(model_counting_sdnf(arg, weights) for arg in sdNNF_formula.args)
    if isinstance(sdNNF_formula, And):
        res = 1.0
        for arg in sdNNF_formula.args:
            res *= model_counting_sdnf(arg, weights)
        return res
    # literal
    if sdNNF_formula in weights:
        return float(weights[sdNNF_formula])
    raise KeyError(f"Manca il peso per {sdNNF_formula}. Attesi sia X che Not(X).")

# ---------------------------
# Helper: full pipeline function
# ---------------------------
def compile_to_sdDNNF(F):
    """
    Full pipeline from arbitrary propositional formula F:
        F -> NNF -> DNNF -> d-DNNF -> sd-DNNF
    """
    F_nnf = to_nnf(F, simplify=True)
    F_dnnf = NNF2DNNF(F_nnf)
    F_ddnnf = DNNF2dDNNF(F_dnnf)
    F_sddnnf = ddnnf2sdNNF(F_ddnnf)
    return simplify(F_sddnnf)


per altri test case -> modifica sezione formula e pesi

In [7]:
# ======= TEST CASE 2 — fatto dal prof a lezione (res = 136) =======

from sympy import symbols, And, Or, Not, S
from sympy.logic.boolalg import to_nnf


# --- FORMULA ---
a, b, c = symbols('a b c')
F = Or(And(a, b), And(c, Not(a)))
print("Formula:", F)

# --- PESI ---
weights = {
    a: 2,   Not(a): 1,
    b: 5,   Not(b): 3,
    c: 7,   Not(c): 1
}

# --- WMC con SMOOTHING NUMERICO su d-DNNF ---
from functools import lru_cache

def _vars(e): return set(e.free_symbols)

@lru_cache(maxsize=None)
def _vars_cached(e):
    return _vars(e)

def wmc_numeric(ddnnf, w):
    """
    WMC su (d-)DNNF con smoothing numerico:
      - literal: w[l]
      - AND:     Π figli
      - OR:      Σ  ( val(figlio) * Π_{v in Vars(OR)\Vars(figlio)} (w[v] + w[~v]) )
    Valido per pesi arbitrari (non normalizzati).
    """
    if ddnnf is S.true:  return 1.0
    if ddnnf is S.false: return 0.0

    # literal
    if ddnnf.is_Symbol:
        return float(w[ddnnf])
    if ddnnf.func is Not and ddnnf.args[0].is_Symbol:
        return float(w[ddnnf])

    # AND
    if ddnnf.func is And:
        out = 1.0
        for ch in ddnnf.args:
            out *= wmc_numeric(ch, w)
        return out

    # OR (numeric smoothing, no tautologies strutturali)
    if ddnnf.func is Or:
        allv = set().union(*(_vars_cached(ch) for ch in ddnnf.args))
        tot = 0.0
        for ch in ddnnf.args:
            missing = allv - _vars_cached(ch)
            factor = 1.0
            for v in missing:
                factor *= (float(w[v]) + float(w[Not(v)]))
            tot += wmc_numeric(ch, w) * factor
        return tot

    raise ValueError(f"Nodo non gestito: {ddnnf}")

# --- COMPILAZIONE FINO A d-DNNF (NO smoothing strutturale) ---
F_nnf  = to_nnf(F, simplify=True)
F_dnnf = NNF2DNNF(F_nnf)
F_dd   = DNNF2dDNNF(F_dnnf)

# --- WMC (d-DNNF, smoothing numerico) ---
wmc_val = wmc_numeric(F_dd, weights)
print("WMC (d-DNNF, smoothing numerico):", wmc_val)

# --- Verifica di riferimento (brute force, stessi pesi reali) ---
"""
  Questa funzione serve a:
    -Verificare che il tuo algoritmo basato su sd-DNNF (o d-DNNF + smoothing numerico) produca lo stesso valore del WMC teorico.
    -Fare da “oracolo” di verità nei test, anche se è inefficiente (cresce esponenzialmente con il numero di variabili).
"""
def brute_force_wmc(F, w):
    vs = list(sorted(F.free_symbols, key=lambda s: s.name))
    tot = 0.0
    n = len(vs)
    for mask in range(1 << n):
        asg = {}
        wgt = 1.0
        for i, v in enumerate(vs):
            val = bool((mask >> i) & 1)
            asg[v] = val
            wgt *= w[v] if val else w[Not(v)]
        if bool(F.subs(asg)):
            tot += wgt
    return tot

ref = brute_force_wmc(F, weights)
print("WMC (brute force):                ", ref)

ok = abs(wmc_val - ref) < 1e-12
print("Confronto:", "OK ✅" if ok else "NO ❌")


Formula: (a & b) | (c & ~a)
WMC (d-DNNF, smoothing numerico): 136.0
WMC (brute force):                 136.0
Confronto: OK ✅


ISTRUZIONI DEBUG



*   n (next) Esegue la riga corrente e si ferma alla successiva (nello stesso livello)

*   s (step) Entra all’interno della funzione chiamata nella riga corrente

* c (continue) Continua l’esecuzione fino al prossimo breakpoint
* r (return) Esegue fino alla fine della funzione corrente e si ferma dopo il return
* l (list) Mostra le righe di codice attorno a quella attuale
* ll: Mostra tutto il corpo della funzione corrente
* u (up) Passa al frame precedente dello stack (chiamante)
* d (down) Passa al frame successivo dello stack (chiamato)


In [ ]:
def shannon_expansion(f, s):

    print("5) entro in shannon_expansion(f,s)")
    result = (s & f.subs(s, True)) | (~s & f.subs(s, False))
    print("shannon", result)
    print("shannon result", result.subs)
    return result.subs({True: True, False: False})

a, b = symbols('{a b}')
f = Or(a, b)

# Espansione rispetto ad 'a':
shannon_expansion(f, a)

5) entro in shannon_expansion(f,s)
shannon {a | (b} & ~{a)
shannon result <bound method Basic.subs of {a | (b} & ~{a)>


{a | (b} & ~{a)

## shannon_expansion(f,s)

La funzione shannon_expansion(f, s) applica l’espansione di Shannon a una formula booleana f rispetto al letterale s, secondo il principio logico per cui una formula può essere riscritta come la disgiunzione di due congiunzioni condizionate sul valore del letterale: (s \land f[s := \text{True}]) \lor (\lnot s \land f[s := \text{False}]). Questo consente di scomporre la formula originaria in due rami distinti: uno in cui s è vero e uno in cui s è falso. Nella funzione, questa trasformazione viene effettuata utilizzando le sostituzioni offerte da SymPy (f.subs(s, True) e f.subs(s, False)) e combinando i risultati con gli operatori logici & (AND), | (OR) e ~ (NOT). Il risultato viene poi restituito con una sostituzione finale {True: True, False: False} per assicurare la corretta interpretazione dei valori booleani nel contesto della libreria. La funzione include anche alcune stampe di debug per mostrare il punto d’ingresso nella funzione e il risultato dell’espansione, sebbene una delle stampe (print("shannon result", result.subs)) contenga un errore e mostri il metodo .subs invece del valore risultante. Questa funzione è particolarmente utile nella trasformazione di formule logiche in DNNF (Decomposable Negation Normal Form) e in tecniche di conteggio dei modelli (#SAT).

In [ ]:
def common(sublist, group):

    print("3) entro in common(sublist, group)")
    if len(sublist) == 0:
        return group
    list_atoms_sets = [s.atoms() - {True, False} for s in group]
    print("list_atoms_sets", list_atoms_sets)
    list_atoms = reduce(lambda s1, s2: s1.union(s2), list_atoms_sets)
    print("list_atoms", list_atoms)
    added = False
    print("sublist", sublist)
    for f in sublist:
        if len(list_atoms.intersection(f.atoms())) > 0:
            group.append(f)
            sublist.remove(f)
            added = True
    if not added:
        return group
    return common(sublist, group)

def common_subs(form):

    print("2) entro in common_subs(form)")
    subs = list(form.args)
    all_groups = []
    while len(subs) > 0:
        sub1 = subs[0]
        group = [sub1]
        subs.remove(sub1)
        group = common(subs, group)
        all_groups.append(group)
    return all_groups

def common_atoms(subformulas):

    print("4) entro in common_atoms(subformulas)")
    subformulas_atoms = [set(filter(lambda a: a not in [True, False], e.atoms())) for e in subformulas]
    print("subformulas_atoms" ,subformulas_atoms)
    common_atoms_set = set()
    for i, sub1 in enumerate(subformulas_atoms):
        for sub2 in subformulas_atoms[i + 1:]:
            common_atoms_set = common_atoms_set.union(sub1.intersection(sub2))
    return common_atoms_set


## Funzioni Common

Le tre funzioni common, common_subs e common_atoms lavorano in sinergia per analizzare la struttura di una formula booleana composta, con l’obiettivo di raggruppare sottoformule che condividono variabili (atomi) comuni.

La funzione common(sublist, group) riceve in input due liste di sottoformule logiche: sublist, che contiene le sottoformule da esaminare, e group, un gruppo iniziale di sottoformule già raccolte. Il suo compito è quello di estendere ricorsivamente il gruppo group aggiungendovi tutte le sottoformule di sublist che condividono almeno un letterale (variabile booleana) con le formule già presenti nel gruppo. Per fare questo, la funzione estrae gli atomi da ogni formula in group, li unisce in un unico insieme list_atoms, e verifica per ogni formula di sublist se vi è intersezione tra i suoi atomi e quelli già presenti nel gruppo. Se sì, la formula viene spostata nel gruppo. Questo processo si ripete finché non si possono aggiungere ulteriori formule, ottenendo un gruppo coerente di sottoformule interconnesse.

La funzione common_subs(form) utilizza common per suddividere i
sottoargomenti (sottoformule) diretti di una formula composta form in gruppi omogenei. Inizia prelevando tutte le sottoformule al primo livello (gli argomenti principali di form), e per ciascuna di esse costruisce un gruppo, che viene poi completato usando la funzione common per includere tutte le altre sottoformule che condividono almeno un atomo con quella iniziale. Alla fine, common_subs restituisce una lista di gruppi di sottoformule logicamente connesse.

Infine, la funzione common_atoms(subformulas) serve a identificare gli atomi (letterali) che sono comuni a più sottoformule. Per ogni formula nella lista subformulas, viene calcolato l’insieme dei suoi atomi escludendo i valori booleani True e False. La funzione quindi confronta ogni coppia di sottoformule e raccoglie l’intersezione tra i loro insiemi di atomi, unendo progressivamente i risultati in un insieme unico common_atoms_set. In questo modo, individua le variabili condivise tra almeno due sottoformule, informazione utile per operazioni come l’espansione di Shannon o la decomposizione logica.

In [ ]:
def NNF2DNNF(nnf):

    print("1) entro in NNF2DNNF(nnf)")
    atoms = list(filter(lambda x: x not in [True, False], nnf.atoms()))
    print("atoms", atoms)
    if len(atoms) <= 1:

        print()
        return nnf.subs({True | False: True, False | True: True, True & False: False, False & True: False})
    all_groups = common_subs(nnf)
    print("all_groups", all_groups)
    result_subformulas = []
    for subformulas in all_groups:
        if len(subformulas) == 1:
            result_subformulas.append(NNF2DNNF(subformulas[0]))
        else:
            args = list(filter(lambda x: x != True, subformulas))
            new_expr = reduce(lambda e1, e2: e1 & e2, args)
            cmn_atoms = common_atoms(subformulas)
            common_atom = cmn_atoms.pop()
            result_subformulas.append(NNF2DNNF(shannon_expansion(new_expr, common_atom)))
    return reduce(lambda e1, e2: e1 & e2, result_subformulas).subs({True | False: True, False | True: True, True & False: False, False & True: False})



## Funzione NNF2DNNF(nnf)

La funzione NNF2DNNF(nnf) ha lo scopo di trasformare una formula logica in Forma Normale di Negazione (NNF) nella sua equivalente Forma Normale di Negazione Decomponibile (DNNF), utilizzando una procedura ricorsiva basata sull’espansione di Shannon e sulla suddivisione strutturata della formula.

All’inizio, la funzione stampa un messaggio di debug e raccoglie tutti gli atomi (cioè le variabili booleane) presenti nella formula nnf, escludendo i valori costanti True e False. Se la formula contiene al più un solo atomo, viene considerata già in forma decomponibile e la funzione la restituisce direttamente, applicando una sostituzione di semplificazione logica (ad esempio True | False → True, True & False → False).

Nel caso in cui siano presenti più atomi, la formula viene suddivisa nei suoi argomenti principali attraverso la funzione common_subs, che raggruppa sottoformule collegate tra loro da variabili comuni. Per ciascun gruppo così ottenuto, se il gruppo contiene solo una formula, questa viene trasformata ricorsivamente tramite NNF2DNNF. Se invece il gruppo contiene più formule, esse vengono congiunte (&) in un’unica espressione, dalla quale viene selezionato un atomo comune tramite la funzione common_atoms. Questo atomo viene usato per applicare l’espansione di Shannon alla formula congiunta, e il risultato viene anch’esso trasformato ricorsivamente in DNNF. Tutti i risultati parziali vengono infine ricombinati tramite l’operatore &, e la formula finale viene semplificata come nei passaggi precedenti.

Nel complesso, NNF2DNNF costruisce una forma decomponibile della formula, assicurandosi che ogni congiunzione combini solo sottoformule su insiemi disgiunti di variabili, un requisito fondamentale per l’efficienza del model counting e della conoscenza compilata in rappresentazioni tipo DNNF.

In [ ]:
def DNNF2dDNNF(dnnf):

    print("entro in DNNF2dDNNF(dnnf)")
    pdb.set_trace()
    atoms = list(filter(lambda x: x not in [True, False], dnnf.atoms()))

    print("atoms DNNF2dDNNF", atoms)
    if dnnf.func == And:
        return reduce(lambda x, y: x & y, deterministic_subs).subs({True | False: True, False | True: True, True & False: False, False & True: False})
    result = reduce(lambda x, y: x | y, deterministic_subs).subs({True | False: True, False | True: True, True & False: False, False & True: False})
    for subexp1, subexp2 in combinations(deterministic_subs, 2):
        form = subexp1 & subexp2
        if satisfiable(form.subs({True: True, False: False})):
            sep = form.atoms().difference({True, False}).pop()
            expanded = shannon_expansion(result, sep).subs({True | False: True, False | True: True, True & False: False, False & True: False})
            return DNNF2dDNNF(expanded)
    return result

## Funzione DNNF2dDNNF(dnnf)

La funzione DNNF2dDNNF(dnnf) ha lo scopo di trasformare una formula in DNNF (Decomposable Negation Normal Form) in una d-DNNF (deterministic DNNF), ovvero una rappresentazione in cui ogni disgiunzione è deterministica, cioè le sue disgiunzioni non possono essere vere contemporaneamente.

All’inizio, la funzione stampa un messaggio di debug e attiva un breakpoint tramite pdb.set_trace() per l’ispezione passo-passo del codice durante l’esecuzione (utile in fase di sviluppo). Subito dopo, estrae tutti gli atomi (variabili booleane) presenti nella formula dnnf, escludendo i valori True e False.

Tuttavia, c’è un errore nel codice: la variabile deterministic_subs non è mai definita né passata come parametro, ma viene usata due volte:
	•	una prima volta nel caso in cui la formula sia una congiunzione (And), applicando la reduce con &
	•	una seconda volta per la disgiunzione (Or) e per analizzare ogni coppia di sottoespressioni

Presumibilmente, deterministic_subs avrebbe dovuto essere qualcosa come list(dnnf.args) o simile, contenente le sottoformule della disgiunzione/congiunzione da elaborare. Correggendo ciò, la funzione dovrebbe:
	1.	Se la formula è una congiunzione (And), restituire la congiunzione dei suoi sottotermini, semplificata.
	2.	Se invece è una disgiunzione (Or), tentare di verificare se le sue disgiunzioni si sovrappongono. Per ogni coppia di sottoformule (subexp1, subexp2), calcola subexp1 & subexp2, e verifica se è soddisfacibile.
	•	Se sì, significa che le due disgiunzioni non sono deterministiche, quindi applica l’espansione di Shannon rispetto a una variabile discriminante (sep), ricavata dagli atomi della loro intersezione.
	•	La formula espansa viene trasformata ricorsivamente con DNNF2dDNNF.

Se nessuna sovrapposizione viene trovata (quindi la formula è già deterministica), restituisce il risultato.

In sintesi, questa funzione cerca di forzare la disgiunzione deterministica usando espansioni di Shannon ogni volta che trova delle sovrapposizioni logiche, con l’obiettivo di ottenere una rappresentazione deterministica utile per ottimizzare operazioni come il model counting. Tuttavia, per funzionare correttamente, va corretta la definizione mancante di deterministic_subs.

In [ ]:
from sympy import symbols, Or, And, Not

def smooth_left(formula):

    print("entro in smooth_left(formula)")
    if isinstance(formula, Or):
        p_set = formula.args[1].free_symbols - formula.args[0].free_symbols
    return formula

def smooth_right(formula):

    print("entro in smooth_right(formula)")
    if isinstance(formula, Or):
        p_set = formula.args[0].free_symbols - formula.args[1].free_symbols
        if p_set:
            p = p_set.pop()
            return Or(formula.args[0], And(formula.args[1], Or(p, Not(p))))
    return formula

def get_subformulas(formula):

    print("entro in get_subformulas(formula)")
    subformulas = set()
    def traverse(node):
        subformulas.add(node)
        for arg in node.args:
            traverse(arg)
    traverse(formula)
    print("traverse ", traverse(formula))
    return list(subformulas)



## Funzioni Smooth Left, Right e get_subformulas

Queste tre funzioni (smooth_left, smooth_right e get_subformulas) sono progettate per lavorare su formule booleane espresse in termini di oggetti di sympy, e servono in particolare a rendere “smooth” le disgiunzioni e a estrarre sottoformule da una formula complessa. Ecco una spiegazione lineare per ciascuna funzione:

⸻

La funzione smooth_left(formula) serve teoricamente a rendere la parte sinistra di una disgiunzione “smooth”, cioè compatibile in termini di variabili con la parte destra. In pratica, controlla se l’input formula è una disgiunzione (Or). Se lo è, calcola l’insieme delle variabili (free_symbols) che compaiono nella seconda parte della disgiunzione (formula.args[1]) ma non nella prima (formula.args[0]), salvandolo in p_set. Tuttavia, nella versione attuale della funzione, questo insieme viene calcolato ma non viene utilizzato per modificare la formula. La funzione restituisce quindi semplicemente la formula originale, rendendo il corpo della funzione inefficace nel suo intento.

La funzione smooth_right(formula) ha invece un comportamento attivo. Dopo aver verificato che la formula sia una disgiunzione (Or), identifica le variabili che compaiono nella prima parte ma non nella seconda (formula.args[0].free_symbols - formula.args[1].free_symbols). Se esistono tali variabili, seleziona una di esse (p) e modifica la parte destra della disgiunzione includendovi un termine “dummy” della forma (p ∨ ¬p) congiunto alla parte destra originaria. Questo termine è logicamente neutro (è sempre vero), ma ha l’effetto di rendere la formula “smooth”, cioè di assicurare che le due parti della disgiunzione condividano le stesse variabili, requisito utile in compilazioni come d-DNNF o sd-DNNF.

Infine, la funzione get_subformulas(formula) ha il compito di raccogliere tutte le sottoformule contenute in una formula logica, comprese le più nidificate. Utilizza una funzione interna traverse per visitare ricorsivamente ogni nodo della formula, aggiungendolo a un insieme subformulas. Dopo aver attraversato l’intera struttura, restituisce tutte le sottoformule come lista. L’invocazione di traverse(formula) all’interno del print() è però superflua, perché la funzione traverse non restituisce nulla; stampa quindi None. Il cuore della funzione è comunque corretto e utile per analisi strutturali della formula.

In [ ]:
def ddnnf2sdNNF(ddnnf_formula):

    print("entro in ddnnf2sdNNF(ddnnf_formula)")
    new_formula = ddnnf_formula
    while True:
        args = get_subformulas(new_formula)
        print("args", args)
        transformed = False
        for i in range(len(args)):
            if isinstance(args[i], Or):
                new_sub = smooth_right(args[i])
                new_sub = smooth_left(new_sub)
                if new_sub != args[i]:
                    new_formula = new_formula.subs(args[i+1], new_sub)
                    transformed = True
        if not transformed or new_formula == formula:
            break
        ddnnf_formula = new_formula
    return new_formula

## Funzione ddnnf2sdNNF(ddnnf_formula)


La funzione **ddnnf2sdNNF(ddnnf_formula)** ha l’obiettivo di trasformare una formula logica in forma d-DNNF (deterministic decomposable negation normal form) nella sua versione sd-DNNF (smooth deterministic DNNF), cioè una forma in cui ogni disgiunzione è non solo deterministica, ma anche smooth, ovvero le disgiunzioni condividono lo stesso insieme di variabili libere (atomi).

Il procedimento si svolge nel seguente modo:

All’inizio, la formula in input ddnnf_formula viene copiata nella variabile new_formula, che sarà progressivamente modificata. La funzione entra poi in un ciclo while True che continuerà fino a che non si verificheranno due condizioni: nessuna trasformazione è più possibile (transformed == False) oppure la formula non cambia più rispetto a una versione precedente.

All’interno del ciclo, si ricavano tutte le sottoformule contenute in new_formula utilizzando la funzione get_subformulas. Successivamente, per ciascuna sottoformula, si verifica se essa è una disgiunzione (Or). Se lo è, la funzione la sottopone prima alla trasformazione smooth_right, poi a smooth_left: entrambe hanno lo scopo di riequilibrare le disgiunzioni in modo che tutti i loro argomenti condividano lo stesso insieme di variabili.

Se la formula risultante (new_sub) è diversa da quella originale (args[i]), allora viene sostituita all’interno della formula complessiva tramite subs, e si imposta transformed = True per indicare che è stata effettuata una modifica.

Il ciclo termina quando nessuna nuova trasformazione è stata effettuata (transformed == False) o quando la formula risultante è uguale alla formula di riferimento (new_formula == formula). Tuttavia, qui c’è un errore: la variabile formula non è definita nel contesto della funzione, e quindi questo confronto causerà un errore in fase di esecuzione. L’intento era probabilmente confrontare con ddnnf_formula o con una copia della formula iniziale.

Alla fine del ciclo, la funzione restituisce new_formula, che rappresenta la versione smooth della formula originale.

In sintesi, ddnnf2sdNNF assicura che tutte le disgiunzioni presenti nella formula rispettino la proprietà di smoothness, fondamentale per alcune tecniche di inferenza e di conteggio dei modelli su rappresentazioni in d-DNNF.


In [ ]:
def model_counting_sdnf(sdNNF_formula, weights):

    print("6) entro in model_counting_sdnf(sdNNF_formula, weights)")
    if isinstance(sdNNF_formula, Or):
        print("or")
        result = 0
        for arg in sdNNF_formula.args:
            result += model_counting_sdnf(arg, weights)
        return result
    elif isinstance(sdNNF_formula, And):
        print("and")
        result = 1
        for arg in sdNNF_formula.args:
            result *= model_counting_sdnf(arg, weights)
        return result
    else:
        print("else")
        if sdNNF_formula in weights:
            return weights[sdNNF_formula]

## Funzione model_counting_sdnf(sdNNF_formula, weights)

La funzione model_counting_sdnf(sdNNF_formula, weights) implementa un algoritmo ricorsivo di model counting per formule booleane espresse in forma sd-DNNF (smooth deterministic decomposable negation normal form). L’obiettivo è calcolare il numero totale di modelli soddisfacenti (o la loro probabilità, se i pesi sono probabilistici) della formula data, utilizzando la sua struttura logica decomponibile.

Ecco una spiegazione lineare della funzione:

La funzione riceve due parametri:
	•	sdNNF_formula: una formula logica già trasformata in sd-DNNF.
	•	weights: un dizionario che associa a ciascun letterale (e alla sua negazione) un valore numerico, tipicamente una probabilità o peso.

All’ingresso, viene stampato un messaggio di debug per indicare che è stata chiamata. Poi, la funzione controlla la struttura della formula:
	1.	Se la formula è una disgiunzione (Or), allora:
	•	Si inizializza result = 0.
	•	Si itera su ciascun argomento della disgiunzione (arg in sdNNF_formula.args).
	•	Per ciascun argomento si richiama ricorsivamente model_counting_sdnf e si somma il risultato.
	•	Questo riflette il fatto che in una disgiunzione deterministica, i modelli dei rami sono mutuamente esclusivi, quindi i conteggi possono essere sommati direttamente.
	2.	Se la formula è una congiunzione (And), allora:
	•	Si inizializza result = 1.
	•	Si moltiplicano i risultati ottenuti ricorsivamente su ciascun argomento.
	•	Questo è corretto perché nella decomposizione, i congiunti usano insiemi di variabili disgiunti, quindi i conteggi possono essere moltiplicati.
	3.	Se la formula è un letterale (caso base dell’albero ricorsivo):
	•	Si verifica se è presente nei pesi (if sdNNF_formula in weights:) e, in tal caso, si restituisce il peso associato (es. 0.5 per una variabile booleana equiprobabile).
	•	Non viene gestito il caso in cui il letterale non sia nel dizionario dei pesi: se ciò accade, la funzione non restituisce nulla (restituirebbe None), il che può causare errori a valle. Per sicurezza, andrebbe aggiunto un else che solleva un’eccezione o restituisce un valore predefinito.

In sintesi, la funzione sfrutta la struttura composizionale della sd-DNNF per calcolare il numero di modelli soddisfacenti in modo efficiente e corretto, combinando somma e prodotto in base alla struttura logica della formula. Questa è una delle principali applicazioni della compilazione logica in DNNF e rappresenta un’alternativa più scalabile rispetto all’enumerazione esplicita dei modelli.

# TEST 1

In [ ]:
#test1

A, B, C = symbols('A B C')
print(A)
print(type(A))
print(B)
print(C)

formula = And(Or(A, B), Or(Not(A), C))  # (A ∨ B) ∧ (¬A ∨ C)

nnf = simplify(formula)
print("\n nnf , ", nnf)
dnnf = NNF2DNNF(nnf)
print("\n dnnf" , dnnf)

weights = {
    A: 0.7, Not(A): 0.3,
    B: 0.6, Not(B): 0.4,
    C: 0.8, Not(C): 0.2,
}

wmc = model_counting_sdnf(dnnf, weights)

print("\n Formula:", formula)
print("DNNF:", dnnf)
print("Weighted Model Count:", wmc)


A
<class 'sympy.core.symbol.Symbol'>
B
C

 nnf ,  (A & C) | (B & ~A)
1) entro in NNF2DNNF(nnf)
atoms [A, C, B]
2) entro in common_subs(form)
3) entro in common(sublist, group)
list_atoms_sets [{A, C}]
list_atoms {A, C}
sublist [B & ~A]
3) entro in common(sublist, group)
all_groups [[A & C, B & ~A]]
4) entro in common_atoms(subformulas)
subformulas_atoms [{A, C}, {A, B}]
5) entro in shannon_expansion(f,s)
shannon False
shannon result <bound method Basic.subs of False>
1) entro in NNF2DNNF(nnf)
atoms []


 dnnf False
6) entro in model_counting_sdnf(sdNNF_formula, weights)
else

 Formula: (A | B) & (C | ~A)
DNNF: False
Weighted Model Count: None


# TEST 2

In [ ]:
from sympy import symbols, And, Or, Not

# Step 1: Definizione dei letterali
a, b, c = symbols('{a b c}')

# Step 2: Formula di esempio
formula = Or(And(a, Not(b)), c)

# Step 3: Conversione in DNNF e poi in d-DNNF
dnnf = NNF2DNNF(formula)
ddnnf = DNNF2dDNNF(dnnf)
sddnnf = ddnnf2sdNNF(ddnnf)

# Step 4: Definizione dei pesi (es. uniformi)
weights = {a: 0.5, Not(a): 0.5, b: 0.5, Not(b): 0.5, c: 0.5, Not(c): 0.5}

# Step 5: Conteggio dei modelli
count = model_counting_sdnf(sddnnf, weights)
print(f"Model count: {count}")

1) entro in NNF2DNNF(nnf)
atoms [b, c}, {a]
2) entro in common_subs(form)
3) entro in common(sublist, group)
list_atoms_sets [{c}}]
list_atoms {c}}
sublist [{a & ~b]
3) entro in common(sublist, group)
all_groups [[c}], [{a & ~b]]
1) entro in NNF2DNNF(nnf)
atoms [c}]

1) entro in NNF2DNNF(nnf)
atoms [b, {a]
2) entro in common_subs(form)
3) entro in common(sublist, group)
list_atoms_sets [{{a}]
list_atoms {{a}
sublist [~b]
3) entro in common(sublist, group)
all_groups [[{a], [~b]]
1) entro in NNF2DNNF(nnf)
atoms [{a]

1) entro in NNF2DNNF(nnf)
atoms [b]

entro in DNNF2dDNNF(dnnf)
> /tmp/ipython-input-988299595.py(5)DNNF2dDNNF()
      3     print("entro in DNNF2dDNNF(dnnf)")
      4     pdb.set_trace()
----> 5     atoms = list(filter(lambda x: x not in [True, False], dnnf.atoms()))
      6 
      7     print("atoms DNNF2dDNNF", atoms)

ipdb> c
atoms DNNF2dDNNF [b, c}, {a]


NameError: name 'deterministic_subs' is not defined

> /tmp/ipython-input-988299595.py(9)DNNF2dDNNF()
      7     print("atoms DNNF2dDNNF", atoms)
      8     if dnnf.func == And:
----> 9         return reduce(lambda x, y: x & y, deterministic_subs).subs({True | False: True, False | True: True, True & False: False, False & True: False})
     10     result = reduce(lambda x, y: x | y, deterministic_subs).subs({True | False: True, False | True: True, True & False: False, False & True: False})
     11     for subexp1, subexp2 in combinations(deterministic_subs, 2):

